In [ ]:
import pandas as pd

import cv2
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# support your google drive in colab
# it's useful to support the kaggle.json credentials
# the code is copied from 
# https://medium.com/@move37timm/using-kaggle-api-for-google-colaboratory-d18645f93648
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files()\
                       .list(q="name = 'kaggle.json'", fields="files(id)")\
                       .execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [ ]:
!kaggle competitions download -c whale-categorization-playground
!unzip train.zip -d data
!unzip test.zip -d data
!ls data

In [ ]:
df = pd.read_csv("train.csv")
df.head()

In [ ]:
counts = df[["Id", "Image"]].groupby("Id").count().sort_values("Image", ascending=False)
print("mean: {}\n\nquantiles:\n{}\n".format(counts.mean(), counts.quantile([0.05, 0.25, 0.5, 0.75, 0.95])))
print("count of items being in excess of 10:\t{}".format(counts.Image[counts.Image > 10].size))

In [ ]:
plt.imshow(cv2.imread("data/train/0014cfdf.jpg")[..., [2, 1, 0]]);

In [ ]:
sns.distplot(df[df.Id != "new_whale"].groupby("Id").size(), 
             bins=50, kde=False, norm_hist=False)